## Imports

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=0
%matplotlib inline

In [ ]:
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import matplotlib.animation as animation
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FormatStrFormatter
from skimage.measure import compare_psnr
import tensorflow as tf
import data.data_selector as ds
from data.dataset import Dataset
import analysis.analysis_picker as ap
import analysis.iso_response_analysis as iso
import utils.plot_functions as pf
import utils.data_processing as dp
import utils.neural_comp_funcs as nc

## Model parameters

In [ ]:
text_width = 506.295 #pt
dpi = 800
fontsize = 12
figsize = (12, 12)

In [ ]:
class lca_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_1280_vh"
    self.version = "5x_0.55"
    self.save_info = "analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

class lca_subspace_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_vh"
    self.version = "5x_4_1.0_0.2"#"3.0"
    self.save_info = "analysis_train"#"analysis_train_kurakin_targeted"
    self.overwrite_analysis_log = False

## Load analysis

In [ ]:
#analysis_params = lca_params()
#analysis_params.project_dir = (os.path.expanduser("~")+"/Work/Projects/")
#analysis_params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+analysis_params.model_name)

analysis_params = lca_subspace_params()
analysis_params.project_dir = (os.path.expanduser("~")+"/Work/ryan_Projects/")
analysis_params.model_dir = (os.path.expanduser("~")+"/Work/ryan_Projects/"+analysis_params.model_name)

analysis_params.do_group_recons = True
analysis_params.do_neuron_visualization = False

analyzer = ap.get_analyzer(analysis_params.model_type)
analyzer.setup(analysis_params)
analyzer.setup_model(analyzer.model_params)
analyzer.load_analysis(save_info=analysis_params.save_info)
analyzer.model_name = analysis_params.model_name

## Iso-Response Contours

In [ ]:
neuron_angles, plot_matrix = analyzer.get_neuron_angles(analyzer.bf_stats)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(neuron_angles, num_bins=50, angle_min=0, angle_max=180,
  y_max=1e6, figsize=(7,7))

In [ ]:
np.min(plot_matrix[plot_matrix>1])

In [ ]:
min_angle = 10
num_above_min = np.count_nonzero(plot_matrix<min_angle) # many angles are -1 or 0
sorted_angle_indices = np.stack(np.unravel_index(np.argsort(plot_matrix.ravel()),
  plot_matrix.shape), axis=1)[num_above_min:, :]

In [ ]:
sorted_angle_indices[0,1]

In [ ]:
bf_id0 = 1274#sorted_angle_indices[14,0]#39
target_neuron_locs = np.argwhere(sorted_angle_indices[:,0] == bf_id0)
low_angle_neuron_ids = np.squeeze(sorted_angle_indices[target_neuron_locs, 1])
bf_id1 = 680#low_angle_neuron_ids[1]

print("BF indices = [",bf_id0,", ",bf_id1,"]")
fig, ax = plt.subplots(2)
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(analyzer.bf_stats["basis_functions"][bf_id0], cmap="Greys_r")
ax[0].set_title(str(bf_id0))
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(analyzer.bf_stats["basis_functions"][bf_id1], cmap="Greys_r")
ax[1].set_title(str(bf_id1))
plt.show()
print("vector angle\t= ", plot_matrix[bf_id0, bf_id1]*(np.pi/180), " rad\n\t\t= ", plot_matrix[bf_id0, bf_id1], " deg")
bf1 = analyzer.bf_stats["basis_functions"][bf_id0].reshape((analyzer.model_params.patch_edge_size**2))
bf2 = analyzer.bf_stats["basis_functions"][bf_id1].reshape((analyzer.model_params.patch_edge_size**2))
bf1_norm = np.linalg.norm(bf1)
bf2_norm = np.linalg.norm(bf2)
print("bf1 norm = ", bf1_norm)
print("bf2 norm = ", bf2_norm)
bf1 /= bf1_norm
bf2 /= bf2_norm

In [ ]:
from data.dataset import Dataset
num_imgs = 2500#10000

x_min = -4.0
x_max = 4.0
y_min = -4.0
y_max = 4.0

x_pts = np.linspace(x_min, x_max, int(np.sqrt(num_imgs)))
y_pts = np.linspace(y_min, y_max, int(np.sqrt(num_imgs)))
X, Y = np.meshgrid(x_pts, y_pts)
proj_datapoints = np.stack([X.reshape(num_imgs), Y.reshape(num_imgs)], axis=1)

proj_matrix, v = analyzer.bf_projections(bf1, bf2)
proj_neuron1 = np.dot(proj_matrix, bf1).T
proj_neuron2 = np.dot(proj_matrix, bf2).T
proj_v = np.dot(proj_matrix, v).T

datapoints = np.stack([np.dot(proj_matrix.T, proj_datapoints[data_id,:]) for data_id in range(num_imgs)]) #inject
datapoints, orig_shape = dp.reshape_data(datapoints, flatten=False)[:2]
datapoints = {"test": Dataset(datapoints, lbls=None, ignore_lbls=None, rand_state=analyzer.rand_state)}
datapoints = analyzer.model.preprocess_dataset(datapoints,
  params={"whiten_data":analyzer.model_params.whiten_data,
  "whiten_method":analyzer.model_params.whiten_method})
datapoints = analyzer.model.reshape_dataset(datapoints, analyzer.model_params)
#datapoints["test"].images /= np.max(np.abs(datapoints["test"].images))
#datapoints["test"].images *= 2.0#analyzer.analysis_params.input_scale

In [ ]:
batch_size = 100
activation_operation=analyzer.model.get_reshaped_group_activity
neuron_index = bf_id0

num_images, data_size = datapoints["test"].images.shape
analyzer.comp_activations = analyzer.compute_activations(datapoints["test"].images, batch_size, activation_operation)
analyzer.comp_activations = analyzer.comp_activations[:, neuron_index]
activity_max = np.amax(np.abs(analyzer.comp_activations))
analyzer.comp_activations = analyzer.comp_activations / (activity_max + 0.00001)
analyzer.comp_activations = analyzer.comp_activations.reshape(int(np.sqrt(num_images)), int(np.sqrt(num_images)))

In [ ]:
num_levels = 10
num_plots_y = 1
num_plots_x = 2
gs1 = gridspec.GridSpec(num_plots_y, num_plots_x, wspace=0.5, width_ratios=[4, 1])
fig = plt.figure(figsize=(6, 6))
curve_ax = pf.clear_axis(fig.add_subplot(gs1[0]))

vmin = np.floor(np.min(analyzer.comp_activations))#0.0
vmax = np.ceil(np.max(analyzer.comp_activations))#1.0
levels = np.linspace(vmin, vmax, num_levels)

#cmap = plt.get_cmap('tab20b')
#cmap = plt.get_cmap('viridis')
#cmap = plt.get_cmap('jet')
cmap = plt.get_cmap("cividis")
cNorm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
scalarMap = matplotlib.cm.ScalarMappable(norm=cNorm, cmap=cmap)

#pts = curve_ax.scatter(proj_datapoints[:,0], proj_datapoints[:,1],
#  vmin=vmin, vmax=vmax, cmap=cmap, c=analyzer.comp_activations.reshape(num_images), s=5.0)
x_pts = np.linspace(x_min/2, x_max/2, int(np.sqrt(num_imgs))) # compress points into smaller area
y_pts = np.linspace(y_min/2, y_max/2, int(np.sqrt(num_imgs)))
X_mesh, Y_mesh = np.meshgrid(x_pts, y_pts)
contsf = curve_ax.contourf(X_mesh, Y_mesh,
  analyzer.comp_activations, levels=levels, vmin=vmin, vmax=vmax, alpha=1.0, antialiased=True, cmap=cmap)


curve_ax.arrow(0, 0, proj_neuron1[0].item(), proj_neuron1[1].item(), width=0.05, head_width=0.15,
  head_length=0.15, fc='k', ec='k')
curve_ax.arrow(0, 0, proj_neuron2[0].item(), proj_neuron2[1].item(), width=0.05, head_width=0.15,
  head_length=0.15, fc='k', ec='k')
#curve_ax.set_title("Angle = "+"{:.2f}".format(neuron_angles[bf_id0, bf_id1])+" deg")
#curve_ax.set_title("Response from pooling neuron "+"{:.0f}".format(pooling_filter_id))
curve_ax.set_ylim([-2, 2.0])
curve_ax.set_xlim([-2, 2.0])
curve_ax.set_aspect("equal")
#cbar = pf.add_colorbar_to_im(pts, aspect=20, pad_fraction=0.5, labelsize=16, ticks=[vmin, vmax])
#cbar.ax.set_yticklabels(["{:.0f}".format(vmin), "{:.0f}".format(vmax)])

gs2 = gridspec.GridSpecFromSubplotSpec(2, 1, gs1[1], wspace=2, hspace=-0.2)
bf1_ax = pf.clear_axis(fig.add_subplot(gs2[0]))
bf1_ax.imshow(analyzer.bf_stats["basis_functions"][bf_id0], cmap="Greys_r")
bf1_ax.set_title("Input\nNeuron {:.0f}".format(bf_id0), color='b')
bf2_ax = pf.clear_axis(fig.add_subplot(gs2[1]))
bf2_ax.imshow(analyzer.bf_stats["basis_functions"][bf_id1], cmap="Greys_r")
bf2_ax.set_title("Input\nNeuron {:.0f}".format(bf_id1), color='k')
#fig.savefig(analyzer.analysis_out_dir+"/vis/l2_neuron_response_contours_pid"+str(pooling_filter_id)+"_bf0id"+str(bf_id0)+"_bf1id"+str(bf_id1)+".png")
plt.show()

In [ ]:
target_neuron_id = 61
in_group_comp_neuron_id = 62
out_group_comp_neuron_id = 170#137#35
x_range = [-2.0, 2.0]
y_range = [-2.0, 2.0]
num_images = int(30**2)
batch_size = 100

target_vector = analyzer.bf_stats["basis_functions"][target_neuron_id]
target_vector = target_vector.reshape(analyzer.model_params.num_pixels)
target_vector = target_vector / np.linalg.norm(target_vector)

in_comparison_vector = analyzer.bf_stats["basis_functions"][in_group_comp_neuron_id]
in_comparison_vector = in_comparison_vector.reshape(analyzer.model_params.num_pixels)
in_comparison_vector = np.squeeze((in_comparison_vector / np.linalg.norm(in_comparison_vector)).T)
out_comparison_vector = analyzer.bf_stats["basis_functions"][out_group_comp_neuron_id]
out_comparison_vector = out_comparison_vector.reshape(analyzer.model_params.num_pixels)
out_comparison_vector = np.squeeze((out_comparison_vector / np.linalg.norm(out_comparison_vector)).T)

analyzer.target_neuron_ids = [target_neuron_id]
analyzer.comparison_neuron_ids = [in_group_comp_neuron_id, out_group_comp_neuron_id]
analyzer.target_vectors = [target_vector]
analyzer.comparison_vectors = [np.stack([in_comparison_vector, out_comparison_vector], axis=0)]

num_comparison_vects = None
use_random_orth_vects = False
analyzer.comp_contour_dataset, datapoints = iso.get_contour_dataset(analyzer, num_comparison_vects,
        use_random_orth_vects, x_range, y_range, num_images)

analyzer.comp_activations = iso.get_normalized_activations(analyzer, datapoints, batch_size,
  activation_operation=analyzer.model.get_reshaped_group_activity)

In [ ]:
analyzer_list = [analyzer]*2
neuron_indices = [0]*2#[0, 0, 0, 0]
orth_indices = [0, 1]
num_levels = 10
show_contours = True

contour_fig, contour_handles = nc.plot_group_iso_contours(analyzer_list, neuron_indices, orth_indices,
  num_levels, x_range, y_range, show_contours, text_width, 1.0, dpi)

In [ ]:
color_vals = dict(zip(["blk", "lt_green", "md_green", "dk_green", "lt_blue", "md_blue", "dk_blue", "lt_red", "md_red", "dk_red"],
  ["#000000", "#A9DFBF", "#196F3D", "#27AE60", "#AED6F1", "#3498DB", "#21618C", "#F5B7B1", "#E74C3C", "#943126"]))

#save_name = "iso_curvature_min85_max95_"
save_name = "iso_curvature_min10_max60_"
run_params = np.load(analyzer.analysis_out_dir+"savefiles/iso_params_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"].item()
min_angle = run_params["min_angle"]
max_angle = run_params["max_angle"]
num_neurons = run_params["num_neurons"]
use_bf_stats = run_params["use_bf_stats"]
analyzer.num_comparison_vects = run_params["num_comparison_vects"]
x_range = run_params["x_range"]
y_range = run_params["y_range"]
num_images = run_params["num_images"]

iso_vectors = np.load(analyzer.analysis_out_dir+"savefiles/iso_vectors_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"].item()
analyzer.target_neuron_ids = iso_vectors["target_neuron_ids"]
analyzer.comparison_neuron_ids = iso_vectors["comparison_neuron_ids"]
analyzer.target_vectors = iso_vectors["target_vectors"]
analyzer.rand_orth_vectors = iso_vectors["rand_orth_vectors"]
analyzer.comparison_vectors = iso_vectors["comparison_vectors"]

analyzer.comp_activations = np.load(analyzer.analysis_out_dir+"savefiles/iso_comp_activations_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"]
analyzer.comp_contour_dataset = np.load(analyzer.analysis_out_dir+"savefiles/iso_comp_contour_dataset_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"].item()
analyzer.rand_activations = np.load(analyzer.analysis_out_dir+"savefiles/iso_rand_activations_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"]
analyzer.rand_contour_dataset = np.load(analyzer.analysis_out_dir+"savefiles/iso_rand_contour_dataset_"+save_name
  +analyzer.analysis_params.save_info+".npz", allow_pickle=True)["data"].item()

In [ ]:
analyzer_list = [analyzer]*analyzer.num_comparison_vects
neuron_indices = [0]*analyzer.num_comparison_vects
orth_indices = list(range(analyzer.num_comparison_vects))
num_levels = 10
show_contours = True

contour_fig, contour_handles = plot_group_iso_contours(analyzer_list, neuron_indices, orth_indices,
  num_levels, x_range, y_range, show_contours, text_width, 1.0, dpi)

## Group invariance reconstructions

In [ ]:
data_dir = os.path.expanduser("~")+"/Work/Datasets/"
data = ds.get_data_from_string("vanhateren", data_dir)

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(np.squeeze(data["train"].images[29, ...]), cmap="Greys_r")
plt.show()

In [ ]:
image_indices = [0, 4, 5, 6, 7, 11, 17, 22, 23, 25]#, 28]
data_subset = {"train": Dataset(data["train"].images[image_indices, ...], lbls=None, ignore_lbls=None,
  rand_state=analyzer.rand_state)}
analyzer.model_params.batch_size = 10
analyzer.model_params.whiten_batch_size = 10
analyzer.model_params.num_patches = 500
data_subset = analyzer.model.preprocess_dataset(data_subset, params=analyzer.model_params)
data_subset = analyzer.model.reshape_dataset(data_subset, analyzer.model_params)

In [ ]:
num_top_images = 4

#num_plot_groups = 5
#select_group_indices = np.random.randint(0, analyzer.model.params.num_groups, num_plot_groups)
#print(select_group_indices)

#[6, 18, 41, 57, 78, 102, 128, 176, 275, 295, 315]
select_group_indices = np.array([6, 41, 57, 18, 176])
num_plot_groups = select_group_indices.size

group_activations, group_angles = analyzer.evaluate_tf_tensor(
  tensor=analyzer.model.get_group_encodings(),
  feed_dict=analyzer.model.get_feed_dict(data_subset["train"].images, is_test=True))
group_activations = group_activations[:, select_group_indices]
new_shape = (analyzer.model_params.num_patches, analyzer.model.params.num_groups,
  analyzer.model.params.num_neurons_per_group)
group_angles = group_angles.reshape(new_shape)[:, select_group_indices, :]

In [ ]:
max_images = []
angle_vectors = []
recons = []
for group_index in range(num_plot_groups):
  data_activity_vector = group_activations[:, group_index]
  max_image_indices = np.argsort(data_activity_vector)
  max_images.append(data_subset["train"].images[max_image_indices[-num_top_images:], ...])
  angle_vectors.append(group_angles[max_image_indices[-num_top_images:], group_index, :])
  sigmas = np.zeros((num_top_images, analyzer.model.params.num_groups))
  sigmas[:, select_group_indices[group_index]] = 1
  zs = np.zeros((num_top_images, analyzer.model.params.num_groups, analyzer.model.params.num_neurons_per_group))
  zs[:, select_group_indices[group_index], :] = angle_vectors[-1]
  input_shape = [num_top_images] + analyzer.model.get_input_shape()[1:]
  feed_dict = analyzer.model.get_feed_dict(np.zeros(input_shape), is_test=True)
  feed_dict[analyzer.sigmas] = sigmas
  feed_dict[analyzer.zs] = zs
  recons.append(analyzer.evaluate_tf_tensor(analyzer.group_recons, feed_dict))
max_images = np.stack(max_images, axis=0)
angle_vectors = np.stack(angle_vectors, axis=0)
recons = np.stack(recons, axis=0)
group_weights = analyzer.evaluate_tf_tensor(analyzer.model.module.group_weights,
  analyzer.model.get_feed_dict(np.zeros(input_shape), is_test=True))

In [ ]:
num_plots_y = recons.shape[0]
num_plots_x = 2 * recons.shape[1] + 1
vmin = np.min(angle_vectors)
vmax = np.max(angle_vectors)
fig = plt.figure(figsize=[figsize[0], figsize[1]/2])
gs = gridspec.GridSpec(num_plots_y, num_plots_x)#, hspace=-0.5, wspace=0.2)
for group_idx in range(num_plots_y):
  group_gs = gridspec.GridSpecFromSubplotSpec(2, 2, gs[group_idx, 0], wspace=-0.01)
  group_order = [0, 1, 3, 2] # clockwise from top-left
  for group_neuron_id in range(analyzer.model.params.num_neurons_per_group):
    weight_ax = pf.clear_axis(fig.add_subplot(group_gs[group_order[group_neuron_id]]))
    weight = group_weights[:, select_group_indices[group_idx], group_neuron_id]
    weight = dp.normalize_data_with_max(weight)[0].reshape([analyzer.model_params.patch_edge_size]*2)
    weight_ax.imshow(weight, cmap="Greys_r", vmin=-1, vmax=1)
  for image_idx, gs_x_idx in enumerate(range(1, num_plots_x, 2)):
    recon_ax = pf.clear_axis(fig.add_subplot(gs[group_idx, gs_x_idx]))
    recon = recons[group_idx, image_idx, ...]
    recon = dp.normalize_data_with_max(recon)[0].reshape([analyzer.model_params.patch_edge_size]*2)
    recon_ax.imshow(recon, cmap="Greys_r", vmin=-1, vmax=1)
    activity_ax = pf.clear_axis(fig.add_subplot(gs[group_idx, gs_x_idx+1]))
    activity_ax.bar(
      range(analyzer.model.params.num_neurons_per_group),
      angle_vectors[group_idx, image_idx, :],
      width=0.8, color="k", align="center")
    activity_ax.set_ylim(vmin, vmax)
    activity_ax.set_yticks(np.linspace(vmin, vmax, 6))
    if gs_x_idx == 1 and group_idx == 0:
      activity_ax.text(-1.6, 0.9, "1", fontsize=fontsize)
      activity_ax.text(-1.6, 0.0, "0", fontsize=fontsize)
      activity_ax.text(-1.6, -0.9, "-1", fontsize=fontsize)
    activity_ax.get_yaxis().set_visible(True)
    activity_ax.yaxis.grid(True, linestyle='--', which='major',
      color='k', alpha=.25)
    yrange = max(activity_ax.get_ylim()) - min(activity_ax.get_ylim())
    xrange = max(activity_ax.get_xlim()) - min(activity_ax.get_xlim())
    aspect = xrange/yrange
    activity_ax.set_aspect(aspect)
plt.show()

fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group_invariance_tiled.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
num_plots_y = recons.shape[0]
num_plots_x = 2 * recons.shape[1] + 1
fig = plt.figure(figsize=[figsize[0], figsize[1]/2])
gs = gridspec.GridSpec(num_plots_y, num_plots_x)#, hspace=-0.5, wspace=0.2)
for group_idx in range(num_plots_y):
  group_gs = gridspec.GridSpecFromSubplotSpec(2, 2, gs[group_idx, 0], wspace=-0.01)
  for group_neuron_id in range(analyzer.model.params.num_neurons_per_group):
    weight_ax = pf.clear_axis(fig.add_subplot(group_gs[group_neuron_id]))
    weight = group_weights[:, select_group_indices[group_idx], group_neuron_id]
    weight = dp.normalize_data_with_max(weight)[0].reshape([analyzer.model_params.patch_edge_size]*2)
    weight_ax.imshow(weight, cmap="Greys_r", vmin=-1, vmax=1)
  for image_idx, gs_x_idx in enumerate(range(1, num_plots_x, 2)):
    recon_ax = pf.clear_axis(fig.add_subplot(gs[group_idx, gs_x_idx]))
    recon = recons[group_idx, image_idx, ...]
    recon = dp.normalize_data_with_max(recon)[0].reshape([analyzer.model_params.patch_edge_size]*2)
    recon_ax.imshow(recon, cmap="Greys_r", vmin=-1, vmax=1)
    inner_gs = gridspec.GridSpecFromSubplotSpec(2, 2, gs[group_idx, gs_x_idx+1])#, wspace=0.05)
    vmin = np.min(angle_vectors)
    vmax = np.max(angle_vectors)
    angle_images = np.ones((analyzer.model_params.num_neurons_per_group,
      analyzer.model_params.patch_edge_size, analyzer.model_params.patch_edge_size))
    for group_neuron_id in range(analyzer.model.params.num_neurons_per_group):
      activity_ax = pf.clear_axis(fig.add_subplot(inner_gs[group_neuron_id]))
      angle_images[group_neuron_id, ...] *= angle_vectors[group_idx, image_idx, group_neuron_id]
      activity_ax.imshow(angle_images[group_neuron_id, ...], cmap="Greys_r", vmin=np.min(angle_vectors), vmax=np.max(angle_vectors))
plt.show()

## Group phase & orientation invariance measurements

In [ ]:
tuning_file_loc = analyzer.analysis_out_dir+"savefiles/group_ot_responses_"+analyzer.analysis_params.save_info+".npz"
ot_grating_responses = np.load(tuning_file_loc, allow_pickle=True)["data"].item()

In [ ]:
group_id = 1

best_or_id = np.argwhere(ot_grating_responses["orientations"] == ot_grating_responses["best_orientations"][group_id, -1]).item()
#or_diffs = ot_grating_responses["orientations"] - ot_grating_responses["orientations"][best_or_id]
num_or = len(ot_grating_responses["orientations"])
or_x_vals = np.linspace(-num_or/2, num_or/2, num_or)

best_ph_id = np.argwhere(ot_grating_responses["phases"] == ot_grating_responses["best_phases"][group_id, -1, best_or_id]).item()
#ph_diffs = ot_grating_responses["phases"] - ot_grating_responses["phases"][best_ph_id]
num_ph = len(ot_grating_responses["phases"])
ph_x_vals = np.linspace(-num_ph/2, num_ph/2, num_ph)

or_resp = np.roll(ot_grating_responses["responses"][group_id, -1, :, best_ph_id], best_or_id)
ph_resp = np.roll(ot_grating_responses["responses"][group_id, -1, best_or_id, :], best_ph_id)

In [ ]:
models = ["ICA", "ISA", "SLCA"]
num_plots_y = len(models)
num_plots_x = 2 # Phase, Orientation

fig = plt.figure(figsize=[figsize[0], figsize[1]])
gs = gridspec.GridSpec(num_plots_y, num_plots_x)#, hspace=-0.5, wspace=0.2)
for y_idx, model_name in enumerate(models):
  ax = fig.add_subplot(gs[y_idx, 0])
  ax.plot(ph_x_vals, ph_resp, color='k')
  #ax.set_xticks(or_x_vals)
  if y_idx == 0:
    ax.set_title("Varying phase", fontsize=fontsize)

  ax = fig.add_subplot(gs[y_idx, 1])
  ax.plot(or_x_vals, or_resp, color='k')
  #ax.set_xticks(ph_x_vals)
  ax.set_ylabel(model_name, rotation="vertical", fontsize=fontsize)
  if y_idx == 0:
    ax.set_title("Varying orientation", fontsize=fontsize)
plt.show()

## Full image recons

In [ ]:
normed_image = (
  (analyzer.full_image - np.min(analyzer.full_image))
  / (np.max(analyzer.full_image) - np.min(analyzer.full_image))).astype(np.float32)

normed_recon = (
  (analyzer.full_recon - np.min(analyzer.full_recon))
  / (np.max(analyzer.full_recon) - np.min(analyzer.full_recon))).astype(np.float32)

fig, ax = plt.subplots(1, 2, figsize=figsize)
ax[0] = pf.clear_axis(ax[0])
ax[0].imshow(np.squeeze(normed_image), cmap="Greys_r")
ax[0].set_title("Input Image", fontsize=fontsize)
ax[1] = pf.clear_axis(ax[1])
ax[1].imshow(np.squeeze(normed_recon), cmap="Greys_r")
percent_active = "{:.2f}".format(analyzer.recon_frac_act*100)
psnr = "{:.2f}".format(compare_psnr(normed_image, normed_recon, data_range=1))
ax[1].set_title("Reconstruction\n"+percent_active+" percent active"+"\n"+"PSNR = "+psnr, fontsize=fontsize)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_image_recon.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

## Group coactivation stats

In [ ]:
data = ds.get_data(analyzer.model_params)
data = analyzer.model.preprocess_dataset(data, analyzer.model_params)
data = analyzer.model.reshape_dataset(data, analyzer.model_params)

In [ ]:
num_imgs = 10000#data["train"].images.shape[0]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=analyzer.model.graph) as sess:
  feed_dict = analyzer.model.get_feed_dict(data["train"].images[0:num_imgs,...])
  sess.run(analyzer.model.init_op, feed_dict)
  analyzer.model.load_full_model(sess, analyzer.analysis_params.cp_loc)
  #run_list = [analyzer.model.a]
  #neuron_activations = sess.run(run_list, feed_dict)
  run_list = [analyzer.model.a, analyzer.model.module.group_activity, analyzer.model.module.group_angles]
  neuron_activations, group_activations, group_angles = sess.run(run_list, feed_dict)

In [ ]:
#cent_activations = neuron_activations - np.mean(neuron_activations)

cent_group_activations = np.zeros_like(group_activations)
for group_id in range(analyzer.model.module.num_groups):
  cent_group_activations[:,group_id] = group_activations[:,group_id] - np.mean(group_activations[:,group_id])

In [ ]:
#cov = 1/num_imgs * np.dot(group_activations.T, group_activations)

cov = 1/num_imgs * np.dot(np.squeeze(cent_group_activations).T, np.squeeze(cent_group_activations))
np.fill_diagonal(cov, 0.0)

#cov = 1/num_imgs * np.dot(np.squeeze(cent_activations).T, np.squeeze(cent_activations))
#np.fill_diagonal(cov, 0.0)

In [ ]:
fig = plt.figure(figsize=figsize)
ax = pf.clear_axis(fig.add_subplot())
im = ax.imshow(cov, cmap="Greys_r")
vmin = np.min(cov)
vmax = np.max(cov)
vmean = vmax - ((vmax - vmin) / 2)
pf.add_colorbar_to_im(im, aspect=20, ticks=[vmin, vmean, vmax], labelsize=fontsize)
ax.set_title("Covariance matrix for group activations", fontsize=fontsize)
#ax.set_title("Covariance matrix for activations", fontsize=fontsize)
plt.show()

fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group_activity_covariance.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)
#fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_activity_covariance.png",
#  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
group_idx = 1
num_bins = 1000
indiv_group_act = group_activations[:, group_idx]
bins = np.linspace(np.min(indiv_group_act), np.max(indiv_group_act), num_bins)
hist, bin_edges = np.histogram(indiv_group_act.flatten(), bins)
bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
bin_centers = bin_left + (bin_right - bin_left)/2

fig, ax = plt.subplots(1, figsize=figsize)
ax.bar(bin_centers, hist, width=2.0, log=True, align="center", color='k')
ax.set_xticks(bin_left, minor=True)
ax.set_xticks(bin_left[::10], minor=False)
ax.xaxis.set_major_formatter(FormatStrFormatter("%0.0f"))
ax.set_xlim([np.min(indiv_group_act), np.max(indiv_group_act)])
vmin = np.min(indiv_group_act)
vmax = np.max(indiv_group_act)
vmean = vmax - ((vmax - vmin)/2)
ax.set_xticks([vmin, vmean, vmax])
ax.set_title("Activity histogram of group "+str(group_idx)+" for "+str(len(indiv_group_act))+" images", fontsize=fontsize)
ax.set_xlabel("Activation", fontsize=fontsize)
ax.set_ylabel("Log Count", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
plt.show()
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_group"+str(group_idx)+"_activity_hist.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
num_images = 500
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
for neuron_id in range(neuron_activations.shape[1]):
  ax.scatter([neuron_id]*num_images, neuron_activations[:num_images, neuron_id], s=0.8, alpha=0.01, color='k')
ax.set_title("Individual Neuron Activity", fontsize=fontsize)
ax.set_xlabel("Neuron Index", fontsize=fontsize)
ax.set_ylabel("Activity", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
plt.show()

## Weight analysis plots

In [ ]:
fig = pf.plot_loc_freq_summary(analyzer.bf_stats, spotsize=9, figsize=[figsize[0], figsize[1]/3], fontsize=fontsize)
fig.savefig(analyzer.analysis_out_dir+"/vis/"+analysis_params.model_name+"_location_frequency_centers.png",
  transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
analyzer.neurn_angles, analyzer.plot_matrix = analyzer.get_neuron_angles(analyzer.bf_stats)

In [ ]:
angle_hist_fig = pf.plot_weight_angle_histogram(analyzer.neuron_angles, num_bins=50, angle_min=0, angle_max=180, figsize=(8,8))
angle_hist_fig.savefig(analyzer.analysis_out_dir+"/vis/neuron_angle_histogram.png")

In [ ]:
weights = analyzer.evals["lca_subspace/weights/w:0"]
weight_indices = np.stack([np.array(id_list) for id_list in analyzer.model.module.group_ids], axis=0)
pooling_weights = np.stack([weights[:, id_list] for id_list in analyzer.model.module.group_ids], axis=1)

In [ ]:
weights = np.reshape(analyzer.evals["lca_subspace/weights/w:0"].T, [analyzer.model.params.num_neurons,
      int(np.sqrt(analyzer.model.params.num_pixels)), int(np.sqrt(analyzer.model.params.num_pixels))])
weight_fig = pf.plot_group_weights(np.squeeze(weights), analyzer.model.module.group_ids,
  title="Dictionary", figsize=(18,18))
weight_fig.savefig(analyzer.analysis_out_dir+"/vis/"+"group_phi.png")

## debugging code for bf_stats spatial freq & orientation

todo: make a wavelett dictionary, compute the bf stats in that